This notebook demos some functionality in ConvoKit to preprocess text, and store the results. In particular, it shows examples of:

* A `TextProcessor` base class that maps per-utterance attributes to per-utterance outputs;
* A `TextParser` class that does dependency parsing;
* Selective and decoupled data storage and loading;
* Per-utterance calls to a transformer;
* Pipelining transformers. 

## Preliminaries: loading an existing corpus.

To start, we load a clean version of a corpus. For speed we will use a 200-utterance subset of the tennis corpus.

In [1]:
import os

In [2]:
os.chdir('../..')

In [3]:
import convokit
from convokit import download

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
# OPTION 1: DOWNLOAD CORPUS 
# UNCOMMENT THESE LINES TO DOWNLOAD CORPUS
# DATA_DIR = '<YOUR DIRECTORY>'
ROOT_DIR = download('tennis-corpus')

# OPTION 2: READ PREVIOUSLY-DOWNLOADED CORPUS FROM DISK
# UNCOMMENT THIS LINE AND REPLACE WITH THE DIRECTORY WHERE THE TENNIS-CORPUS IS LOCATED
# ROOT_DIR = '<YOUR DIRECTORY>'

corpus = convokit.Corpus(ROOT_DIR, utterance_end_index=199)

Dataset already exists at /Users/calebchiam/.convokit/downloads/tennis-corpus


In [5]:
corpus.print_summary_stats()

Number of Users: 9
Number of Utterances: 200
Number of Conversations: 100


In [6]:
# SET YOUR OWN OUTPUT DIRECTORY HERE. 
OUT_DIR = '<YOUR OUTPUT DIRECTORY>'

Here's an example of an utterance from this corpus (questions asked to tennis players after matches, and the answers they give):

In [7]:
test_utt_id = '1681_14.a'
utt = corpus.get_utterance(test_utt_id)

In [8]:
utt.text

"Yeah, but many friends went with me, Japanese guy. So I wasn't -- I wasn't like homesick. But now sometimes I get homesick."

Right now, `utt.meta` contains the following fields:

In [9]:
utt.meta

{'is_answer': True, 'is_question': False, 'pair_idx': '1681_14'}

## The TextProcessor class

Many of our transformers are per-utterance mappings of one attribute of an utterance to another. To facilitate these calls, we use a `TextProcessor` class that inherits from `Transformer`. 

`TextProcessor` is initialized with the following arguments:

* `proc_fn`: the mapping function. Supports one of two function signatures: `proc_fn(input)` and `proc_fn(input, auxiliary_info)`. 
* `input_field`: the attribute of the utterance that `proc_fn` will take as input. If set to `None`, will default to reading `utt.text`, as seems to be presently done.
* `output_field`: the name of the attribute that the output of `proc_fn` will be written to. 
* `aux_input`: any auxiliary input that `proc_fn` needs (e.g., a pre-loaded model); passed in as a dict.
* `input_filter`: a boolean function of signature `input_filter(utterance, aux_input)`, where `aux_input` is again passed as a dict. If this returns `False` then the particular utterance will be skipped; by default it will always return `True`.

Both `input_field` and `output_field` support multiple items -- that is, `proc_fn` could take in multiple attributes of an utterance and output multiple attributes. I'll show how this works in advanced usage, below.

"Attribute" is a deliberately generic term. `TextProcessor` could produce "features" as we may conventionally think of them (e.g., wordcount, politeness strategies). It can also be used to pre-process text, i.e., generate alternate representations of the text. 

In [10]:
from convokit.text_processing import TextProcessor

### simple example: cleaning the text

As a simple example, suppose we want to remove hyphens "`--`" from the text as a preprocessing step. To use `TextProcessor` to do this for us, we'd define the following as a `proc_fn`:

In [11]:
def preprocess_text(text):
    text = text.replace(' -- ', ' ')
    return text

Below, we initialize `prep`, a `TextProcessor` object that will run `preprocess_text` on each utterance.

When we call `prep.transform()`, the following will occur:

* Because we didn't specify an input field, `prep` will pass `utterance.text` into `preprocess_text`
* It will write the output -- the text minus the hyphens -- to a field called `clean_text` that will be stored in the utterance meta and that can be accessed as `utt.meta['clean_text']` or `utt.get_info('clean_text')`

In [12]:
prep = TextProcessor(proc_fn=preprocess_text, output_field='clean_text')
corpus = prep.transform(corpus)

And as desired, we now have a new field attached to `utt`.

In [13]:
utt.get_info('clean_text')

"Yeah, but many friends went with me, Japanese guy. So I wasn't I wasn't like homesick. But now sometimes I get homesick."

## Parsing text with the TextParser class

One common utterance-level thing we want to do is parse the text. In practice, in increasing order of (computational) difficulty, this typically entails:

* proper tokenizing of words and sentences;
* POS-tagging;
* dependency-parsing. 

As such, we provide a `TextParser` class that inherits from `TextProcessor` to do all of this, taking in the following arguments:

* `output_field`: defaults to `'parsed'`
* `input_field`
* `mode`: whether we want to go through all of the above steps (which may be expensive) or stop mid-way through. Supports the following options: `'tokenize'`, `'tag'`, `'parse'` (the default).

Under the surface, `TextParser` actually uses two separate models: a `spacy` object that does word tokenization, tagging and parsing _per sentence_, and `nltk`'s sentence tokenizer. The rationale is:

* `spacy` doesn't support sentence tokenization without dependency-parsing, and we often want sentence tokenization without having to go through the effort of parsing.
* We want to be consistent (as much as possible, given changes to spacy and nltk) in the tokenizations we produce, between runs where we don't want parsing and runs where we do.

If we've pre-loaded these models, we can pass them into the constructor too, as:

* `spacy_nlp`
* `sent_tokenizer`

In [14]:
from convokit.text_processing import TextParser

In [15]:
parser = TextParser(input_field='clean_text', verbosity=50)

In [16]:
corpus = parser.transform(corpus)

050/200 utterances processed
100/200 utterances processed
150/200 utterances processed
200/200 utterances processed


### parse output

A parse produced by `TextParser` is serialized in text form. It is a list consisting of sentences, where each sentence is a dict with

* `toks`: a list of tokens (i.e., words) in the sentence;
* `rt`: the index of the root of the dependency tree (i.e., `sentence['toks'][sentence['rt']` gives the root)

Each token, in turn, contains the following:

* `tok`: the text of the token;
* `tag`: the tag;
* `up`: the index of the parent of the token in the dependency tree (no entry for the root);
* `down`: the indices of the children of the token;
* `dep`: the dependency of the edge between the token and its parent.

In [17]:
test_parse = utt.get_info('parsed')

In [18]:
test_parse[0]

{'rt': 5,
 'toks': [{'tok': 'Yeah', 'tag': 'UH', 'dep': 'intj', 'up': 5, 'dn': []},
  {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
  {'tok': 'but', 'tag': 'CC', 'dep': 'cc', 'up': 5, 'dn': []},
  {'tok': 'many', 'tag': 'JJ', 'dep': 'amod', 'up': 4, 'dn': []},
  {'tok': 'friends', 'tag': 'NNS', 'dep': 'nsubj', 'up': 5, 'dn': [3, 10]},
  {'tok': 'went', 'tag': 'VBD', 'dep': 'ROOT', 'dn': [0, 1, 2, 4, 6, 8, 11]},
  {'tok': 'with', 'tag': 'IN', 'dep': 'prep', 'up': 5, 'dn': [7]},
  {'tok': 'me', 'tag': 'PRP', 'dep': 'pobj', 'up': 6, 'dn': []},
  {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
  {'tok': 'Japanese', 'tag': 'JJ', 'dep': 'amod', 'up': 10, 'dn': []},
  {'tok': 'guy', 'tag': 'NN', 'dep': 'appos', 'up': 4, 'dn': [9]},
  {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 5, 'dn': []}]}

If we didn't want to go through the trouble of dependency-parsing (which could be expensive) we could initialize `TextParser` with `mode='tag'`, which only POS-tags tokens:

In [19]:
texttagger = TextParser(output_field='tagged', input_field='clean_text', mode='tag')
corpus = texttagger.transform(corpus)

In [20]:
utt.get_info('tagged')[0]

{'toks': [{'tok': 'Yeah', 'tag': 'UH'},
  {'tok': ',', 'tag': ','},
  {'tok': 'but', 'tag': 'CC'},
  {'tok': 'many', 'tag': 'JJ'},
  {'tok': 'friends', 'tag': 'NNS'},
  {'tok': 'went', 'tag': 'VBD'},
  {'tok': 'with', 'tag': 'IN'},
  {'tok': 'me', 'tag': 'PRP'},
  {'tok': ',', 'tag': ','},
  {'tok': 'Japanese', 'tag': 'JJ'},
  {'tok': 'guy', 'tag': 'NN'},
  {'tok': '.', 'tag': '.'}]}

## Storing and loading corpora

We've now computed a bunch of utterance-level attributes. 

In [21]:
list(utt.meta.keys())

['is_answer', 'is_question', 'pair_idx', 'clean_text', 'parsed', 'tagged']

By default, calling `corpus.dump` will write all of these attributes to disk, within the file that stores utterances; later calling `corpus.load` will load all of these attributes back into a new corpus. For big objects like parses, this incurs a high computational burden (especially if in a later use case you might not even need to look at parses).

To avoid this, `corpus.dump`  takes an optional argument `fields_to_skip`, which is a dict of object type (`'utterance'`, `'conversation'`, `'user'`, `'corpus'`) to a list of fields that we do not want to write to disk. 

The following call will write the corpus to disk, without any of the preprocessing output we generated above:

In [22]:
corpus.dump(os.path.basename(OUT_DIR), base_path=os.path.dirname(OUT_DIR), 
            fields_to_skip={'utterance': ['parsed','tagged','clean_text']})

For attributes we want to keep around, but that we don't want to read and write to disk in a big batch with all the other corpus data, `corpus.dump_info` will dump fields of a Corpus object into separate files. This takes the following arguments as input:

* `obj_type`: which type of Corpus object you're dealing with.
* `fields`: a list of the fields to write. 
* `dir_name`: which directory to write to; by default will write to the directory you read the corpus from.

This function will write each field in `fields` to a separate file called `info.<field>.jsonl` where each line of the file is a json-serialized dict: `{"id": <ID of object>, "value": <object.get_info(field)>}`. 

In [23]:
corpus.dump_info('utterance',['parsed','tagged'], dir_name = OUT_DIR)

As expected, we now have the following files in the output directory:

In [ ]:
ls $OUT_DIR

If we now initialize a new corpus by reading from this directory:

In [24]:
new_corpus = convokit.Corpus(OUT_DIR)

In [25]:
new_utt = new_corpus.get_utterance(test_utt_id)

We see that things that we've omitted in the `corpus.dump` call will not be read.

In [26]:
new_utt.meta.keys()

KeysView({'is_answer': True, 'is_question': False, 'pair_idx': '1681_14'})

As a counterpart to `corpus.dump_info` we can also load auxiliary information on-demand. Here, this call will look for `info.<field>.jsonl` in the directory of `new_corpus` (or an optionally-specified `dir_name`) and attach the value specified in each line of the file to the utterance with the associated id:

In [27]:
new_corpus.load_info('utterance',['parsed'])

In [30]:
new_utt.get_info('parsed')

[{'rt': 5,
  'toks': [{'tok': 'Yeah', 'tag': 'UH', 'dep': 'intj', 'up': 5, 'dn': []},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
   {'tok': 'but', 'tag': 'CC', 'dep': 'cc', 'up': 5, 'dn': []},
   {'tok': 'many', 'tag': 'JJ', 'dep': 'amod', 'up': 4, 'dn': []},
   {'tok': 'friends', 'tag': 'NNS', 'dep': 'nsubj', 'up': 5, 'dn': [3, 10]},
   {'tok': 'went', 'tag': 'VBD', 'dep': 'ROOT', 'dn': [0, 1, 2, 4, 6, 8, 11]},
   {'tok': 'with', 'tag': 'IN', 'dep': 'prep', 'up': 5, 'dn': [7]},
   {'tok': 'me', 'tag': 'PRP', 'dep': 'pobj', 'up': 6, 'dn': []},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
   {'tok': 'Japanese', 'tag': 'JJ', 'dep': 'amod', 'up': 10, 'dn': []},
   {'tok': 'guy', 'tag': 'NN', 'dep': 'appos', 'up': 4, 'dn': [9]},
   {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 5, 'dn': []}]},
 {'rt': 2,
  'toks': [{'tok': 'So', 'tag': 'RB', 'dep': 'advmod', 'up': 2, 'dn': []},
   {'tok': 'I', 'tag': 'PRP', 'dep': 'nsubj', 'up': 2, 'dn': []},
   {'to

## Per-utterance calls

`TextProcessor` objects also support calls per-utterance via `TextProcessor.transform_utterance()`. These calls take in raw strings as well as utterances, and will return an utterance:



In [28]:
test_str = "I played -- a tennis match."

In [29]:
prep.transform_utterance(test_str)

Utterance({'obj_type': 'utterance', '_owner': None, 'meta': {'clean_text': 'I played a tennis match.'}, '_id': None, 'user': None, 'root': None, 'reply_to': None, 'timestamp': None, 'text': 'I played -- a tennis match.'})

In [30]:
from convokit.model import Utterance
adhoc_utt = Utterance(text=test_str)

In [31]:
adhoc_utt = prep.transform_utterance(adhoc_utt)

In [32]:
adhoc_utt.get_info('clean_text')

'I played a tennis match.'

## Pipelines

Finally, we can string together multiple transformers, and hence `TextProcessors`, into a pipeline, using a `ConvokitPipeline` object. This is analogous to (and in fact inherits from) scikit-learn's `Pipeline` class. 

In [33]:
from convokit.convokitPipeline import ConvokitPipeline

As an example, suppose we want to both clean the text and parse it. We can chain the required steps to get there by initializing `ConvokitPipeline` with a list of steps, represented as a tuple of `(<step name>, initialized transformer-like object)`:

* `'prep'`, our de-hyphenator
* `'parse'`, our parser


In [34]:
parse_pipe = ConvokitPipeline([('prep', TextProcessor(preprocess_text, 'clean_text_pipe')),
                ('parse', TextParser('parsed_pipe', input_field='clean_text_pipe',
                                    verbosity=50))])

In [35]:
corpus = parse_pipe.transform(corpus)

050/200 utterances processed
100/200 utterances processed
150/200 utterances processed
200/200 utterances processed


In [36]:
utt.get_info('parsed_pipe')

[{'rt': 5,
  'toks': [{'tok': 'Yeah', 'tag': 'UH', 'dep': 'intj', 'up': 5, 'dn': []},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
   {'tok': 'but', 'tag': 'CC', 'dep': 'cc', 'up': 5, 'dn': []},
   {'tok': 'many', 'tag': 'JJ', 'dep': 'amod', 'up': 4, 'dn': []},
   {'tok': 'friends', 'tag': 'NNS', 'dep': 'nsubj', 'up': 5, 'dn': [3, 10]},
   {'tok': 'went', 'tag': 'VBD', 'dep': 'ROOT', 'dn': [0, 1, 2, 4, 6, 8, 11]},
   {'tok': 'with', 'tag': 'IN', 'dep': 'prep', 'up': 5, 'dn': [7]},
   {'tok': 'me', 'tag': 'PRP', 'dep': 'pobj', 'up': 6, 'dn': []},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 5, 'dn': []},
   {'tok': 'Japanese', 'tag': 'JJ', 'dep': 'amod', 'up': 10, 'dn': []},
   {'tok': 'guy', 'tag': 'NN', 'dep': 'appos', 'up': 4, 'dn': [9]},
   {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 5, 'dn': []}]},
 {'rt': 2,
  'toks': [{'tok': 'So', 'tag': 'RB', 'dep': 'advmod', 'up': 2, 'dn': []},
   {'tok': 'I', 'tag': 'PRP', 'dep': 'nsubj', 'up': 2, 'dn': []},
   {'to

As promised, the pipeline also works to transform utterances.

In [37]:
test_utt = parse_pipe.transform_utterance(test_str)

In [38]:
test_utt.get_info('parsed_pipe')

[{'rt': 1,
  'toks': [{'tok': 'I', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []},
   {'tok': 'played', 'tag': 'VBD', 'dep': 'ROOT', 'dn': [0, 4, 5]},
   {'tok': 'a', 'tag': 'DT', 'dep': 'det', 'up': 4, 'dn': []},
   {'tok': 'tennis', 'tag': 'NN', 'dep': 'compound', 'up': 4, 'dn': []},
   {'tok': 'match', 'tag': 'NN', 'dep': 'dobj', 'up': 1, 'dn': [2, 3]},
   {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 1, 'dn': []}]}]

### Some advanced usage: playing around with parameters

The point of the following is to demonstrate more elaborate calls to `TextProcessor`. As an example, we will count words in an utterance.

First, we'll initialize a `TextProcessor` that does wordcounts (i.e., `len(x.split())`) on just the raw text (`utt.text`), writing output to field `wc_raw`.

In [39]:
wc_raw = TextProcessor(proc_fn=lambda x: len(x.split()), output_field='wc_raw')
corpus = wc_raw.transform(corpus)

In [40]:
utt.get_info('wc_raw')

23

If we instead wanted to wordcount our preprocessed text, with the hyphens removed, we can specify `input_field='clean_text'` -- as such, the `TextProcessor` will read from `utt.get_info('clean_text')` instead. 

In [41]:
wc = TextProcessor(proc_fn=lambda x: len(x.split()), output_field='wc', input_field='clean_text')
corpus = wc.transform(corpus)

Here we see that we are no longer counting the extra hyphen.

In [42]:
utt.get_info('wc')

22

Likewise, we can count characters:

In [43]:
chars = TextProcessor(proc_fn=lambda x: len(x), output_field='ch', input_field='clean_text')
corpus = chars.transform(corpus)

In [44]:
utt.get_info('ch')

120

Suppose that for some reason we now wanted to calculate:

* characters per word
* words per character (the reciprocal)

This requires:

* a `TextProcessor` that takes in multiple input fields, `'ch'` and `'wc'`;
* and that writes to multiple output fields, `'char_per_word'` and `'word_per_char'`.

Here's how the resultant object, `char_per_word`, handles this:

* in `transform()`, we pass `proc_fn` a dict mapping input field name to value, e.g., `{'wc': 22, 'ch': 120}`
* `proc_fn` will be written to return a tuple, where each element of that tuple corresponds to each element of the list we've passed to `output_field`, e.g., 

```out0, out1 = proc_fn(input)
utt.set_info('char_per_word', out0) 
utt.set_info('word_per_char', out1)```

In [45]:
char_per_word = TextProcessor(proc_fn=lambda x: (x['ch']/x['wc'], x['wc']/x['ch']), 
                              output_field=['char_per_word', 'word_per_char'], input_field=['ch','wc'])
corpus = char_per_word.transform(corpus)

In [46]:
utt.get_info('char_per_word')

5.454545454545454

In [47]:
utt.get_info('word_per_char')

0.18333333333333332

### Some advanced usage: input filters

Just for the sake of demonstration, suppose we wished to save some computation time and only parse the questions in a corpus. We can do this by specifying `input_filter` (which, recall discussion above, takes as argument an `Utterance` object). 

In [48]:
def is_question(utt, aux={}):
    return utt.meta['is_question']

In [49]:
qparser = TextParser(output_field='qparsed', input_field='clean_text', input_filter=is_question, verbosity=50)

In [50]:
corpus = qparser.transform(corpus)

050/200 utterances processed
100/200 utterances processed
150/200 utterances processed
200/200 utterances processed


Since our test utterance is not a question, `qparser.transform()` will skip over it, and hence the utterance won't have the 'qparsed' attribute (and `get_info` returns `None`):

In [51]:
utt.get_info('qparsed')

However, if we take an utterance that's a question, we see that it is indeed parsed:

In [52]:
q_utt_id = '1681_14.q'
q_utt = corpus.get_utterance(q_utt_id)
q_utt.text

'How hard was it for you when, 13 years, left your parents, left Japan to go to the States. Was it a big step for you?'

In [53]:
q_utt.get_info('qparsed')

[{'rt': 2,
  'toks': [{'tok': 'How', 'tag': 'WRB', 'dep': 'advmod', 'up': 1, 'dn': []},
   {'tok': 'hard', 'tag': 'JJ', 'dep': 'acomp', 'up': 2, 'dn': [0]},
   {'tok': 'was', 'tag': 'VBD', 'dep': 'ROOT', 'dn': [1, 3, 4, 11, 22]},
   {'tok': 'it', 'tag': 'PRP', 'dep': 'nsubj', 'up': 2, 'dn': []},
   {'tok': 'for', 'tag': 'IN', 'dep': 'prep', 'up': 2, 'dn': [5]},
   {'tok': 'you', 'tag': 'PRP', 'dep': 'pobj', 'up': 4, 'dn': []},
   {'tok': 'when', 'tag': 'WRB', 'dep': 'advmod', 'up': 11, 'dn': []},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 11, 'dn': []},
   {'tok': '13', 'tag': 'CD', 'dep': 'nummod', 'up': 9, 'dn': []},
   {'tok': 'years', 'tag': 'NNS', 'dep': 'npadvmod', 'up': 11, 'dn': [8]},
   {'tok': ',', 'tag': ',', 'dep': 'punct', 'up': 11, 'dn': []},
   {'tok': 'left',
    'tag': 'VBD',
    'dep': 'advcl',
    'up': 2,
    'dn': [6, 7, 9, 10, 13, 14, 15]},
   {'tok': 'your', 'tag': 'PRP$', 'dep': 'poss', 'up': 13, 'dn': []},
   {'tok': 'parents', 'tag': 'NNS', 'dep': 'dobj